## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("weatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Albany,US,42.6001,-73.9662,16.45,45,33,1.01,scattered clouds
1,1,Sechura,PE,-5.5569,-80.8222,80.42,57,93,13.56,overcast clouds
2,2,Jamestown,US,42.0970,-79.2353,14.52,35,0,12.66,clear sky
3,3,Cockburn Town,TC,21.4612,-71.1419,78.76,73,40,6.91,scattered clouds
4,4,Vaini,TO,-21.2000,-175.2000,80.33,69,94,15.28,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 87


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Sechura,PE,-5.5569,-80.8222,80.42,57,93,13.56,overcast clouds
3,3,Cockburn Town,TC,21.4612,-71.1419,78.76,73,40,6.91,scattered clouds
4,4,Vaini,TO,-21.2000,-175.2000,80.33,69,94,15.28,overcast clouds
14,14,Saint-Philippe,RE,-21.3585,55.7679,80.15,85,98,13.06,light rain
16,16,Luanda,AO,-8.8368,13.2343,77.11,86,73,3.69,broken clouds
17,17,Georgetown,MY,5.4112,100.3354,77.90,93,40,5.75,scattered clouds
18,18,Kapaa,US,22.0752,-159.3190,77.47,63,0,10.36,clear sky
19,19,Arraial Do Cabo,BR,-22.9661,-42.0278,78.76,83,20,9.22,few clouds
25,25,Robertsport,LR,6.7533,-11.3686,80.20,84,44,8.70,scattered clouds
29,29,Faanui,PF,-16.4833,-151.7500,80.80,76,100,16.51,light rain


In [5]:
# 4a. Determine if there are any empty rows.
np.where(pd.isnull(preferred_cities_df))

(array([], dtype=int64), array([], dtype=int64))

In [13]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna()
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Sechura,PE,-5.5569,-80.8222,80.42,57,93,13.56,overcast clouds
3,3,Cockburn Town,TC,21.4612,-71.1419,78.76,73,40,6.91,scattered clouds
4,4,Vaini,TO,-21.2000,-175.2000,80.33,69,94,15.28,overcast clouds
14,14,Saint-Philippe,RE,-21.3585,55.7679,80.15,85,98,13.06,light rain
16,16,Luanda,AO,-8.8368,13.2343,77.11,86,73,3.69,broken clouds
17,17,Georgetown,MY,5.4112,100.3354,77.90,93,40,5.75,scattered clouds
18,18,Kapaa,US,22.0752,-159.3190,77.47,63,0,10.36,clear sky
19,19,Arraial Do Cabo,BR,-22.9661,-42.0278,78.76,83,20,9.22,few clouds
25,25,Robertsport,LR,6.7533,-11.3686,80.20,84,44,8.70,scattered clouds
29,29,Faanui,PF,-16.4833,-151.7500,80.80,76,100,16.51,light rain


In [15]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Sechura,PE,80.42,overcast clouds,-5.5569,-80.8222,
3,Cockburn Town,TC,78.76,scattered clouds,21.4612,-71.1419,
4,Vaini,TO,80.33,overcast clouds,-21.2000,-175.2000,
14,Saint-Philippe,RE,80.15,light rain,-21.3585,55.7679,
16,Luanda,AO,77.11,broken clouds,-8.8368,13.2343,
17,Georgetown,MY,77.90,scattered clouds,5.4112,100.3354,
18,Kapaa,US,77.47,clear sky,22.0752,-159.3190,
19,Arraial Do Cabo,BR,78.76,few clouds,-22.9661,-42.0278,
25,Robertsport,LR,80.20,scattered clouds,6.7533,-11.3686,
29,Faanui,PF,80.80,light rain,-16.4833,-151.7500,


In [16]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
            hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
            print("Hotel not found.... skipping.")
        

Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.
Hotel not found.... skipping.


In [25]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna()

hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Sechura,PE,80.42,overcast clouds,-5.5569,-80.8222,Hospedaje Costa Bella
3,Cockburn Town,TC,78.76,scattered clouds,21.4612,-71.1419,Osprey Beach Hotel
4,Vaini,TO,80.33,overcast clouds,-21.2000,-175.2000,Keleti Beach Resort
14,Saint-Philippe,RE,80.15,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
16,Luanda,AO,77.11,broken clouds,-8.8368,13.2343,EPIC SANA Luanda


In [26]:
# 8a. Create the output File (CSV)
output_data_file = "weatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [27]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Description</dt><dd>{Current Description}
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [34]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(15.0, 16.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=3)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))